In [1]:
import os, datetime
import pandas as pd

# change these
HOURS_DIFFERENTIAL = 6 # have to subtract 6

# jump is good, not sure on others
list_of_finals = ['round_fallmountain', 'round_hex', 'round_royalfumble', 'round_jump_showdown']

with open('totalshows.txt') as f:
    total_shows = f.read()
total_shows = int(total_shows.strip())


# if data folder doesn't exist, make it
if not os.path.exists('data'):
    os.makedirs('data')

In [2]:
# Notes

# find player id for round - don't think we can do anything about that
# use that as well instead of username
# only affects userEndRoundLines

# check elim time for jump_showdown - seems as though personal end times are only for...
# ...finishing races early

# figure out if there's a max number of shows that will be stored in the .log : seems the whole session

In [3]:
def cleanLines(lines):
    return [line.replace('[', '').replace(']', '').replace('>', '').strip() for line in lines]

# splits show into show data and rounds
def roundSplit(lines):
    lines = cleanLines(lines)
    splits = []
    # start of each round in highlights section
    for i, line in enumerate(lines):
        if 'Round' in line:
            splits.append(i)
    
    splits.append(0)
    # split it into rounds
    rounds = [lines[splits[i-1]:splits[i]] if i != len(splits)-1 else lines[splits[i-1]:]
              for i in range(1, len(splits))]
    
    return lines[1:splits[0]], rounds

# takes registered and connected lines
# returns start times for rounds
# (time at which game is found)
def getStartTimes(reg, conne):
    startTimes = []
    
    for i in range(len(conne)):
        registeredTime = datetime.datetime.strptime(reg[i].split('at: ')[-1], '%m/%d/%Y %H:%M:%S %p')
        connectedTime = datetime.datetime.strptime(conne[i].split(': [')[0], '%H:%M:%S.%f')
        
        d = (connectedTime - registeredTime)
        startTimes.append(registeredTime + (d - datetime.timedelta(days=d.days, hours=HOURS_DIFFERENTIAL)))
    
    return startTimes

def getTimeTaken(start, end):
    d = datetime.datetime.strptime(end, '%H:%M:%S.%f') - start
    # start time was already adjusted for HOURS_DIFFERENTIAL, so need to again
    return str(d - datetime.timedelta(days=d.days, hours=HOURS_DIFFERENTIAL))[2:] # so hours isn't included

# edit this later change
# grab the last Seasonthing in txt file - countdown in .log
# if its larger than the last one, then the season reset
def getSeason():
    return 2

# gets lines for CompletedEpisode section of a show
def getShowLines(lines, marker):
    finalLines = []
    tempLines = lines[marker:]
    for line in tempLines:
        if line == '':
            continue
        if '>' not in line and '[Round' not in line and '[Complet' not in line:
            break
        if line != "":
            finalLines.append(line)
    return finalLines

# get map info lines (qual percent and # participants)
def getExtraRoundInfoLines(possLines): # rename?
    rnds = []
    currRnd = possLines[0][1].split()[0]
    currSID = possLines[0][0]

    # get last line before map switches (and server ID)
    for i, (serverID, line) in enumerate(possLines):
        rnd = line.split()[0]
        if rnd != currRnd: 
            currRnd = rnd
            currSID = serverID
            rnds.append(possLines[i-1][1])
        elif serverID != currSID:
            currRnd = rnd
            currSID = serverID
            rnds.append(possLines[i-1][1])


    rnds.append(possLines[-1][1])      
    return rnds


# preprocessGrade1 has been retired
# remove lines in-between completed games
def preprocessGrade2(lines):
    print(len(lines))
    lines2 = []
    allGood = False
    badgeID = False
    for line in lines:
        if '[CATAPULT] Login Succeeded' in line:
            allGood = True
        if 'BadgeId:' in line:
            badgeID = True
        if badgeID:
            if '[ClientGameManager] Shutdown' in line:
                badgeID = False
                allGood = False

        if allGood:
            lines2.append(line)
    lines = lines2
    print(len(lines))
    return lines

# helper function (to account for if round wraps around midnight)
def subtractHours(a, b):
    c = (datetime.datetime.strptime(b, '%H:%M:%S.%f') - datetime.datetime.strptime(a, '%H:%M:%S.%f'))
    return str(c - datetime.timedelta(days=c.days))

# save data in csvs
def saveData(show, roundsList):
    new_shows_df = pd.DataFrame(pd.Series(show)).T
    new_rounds_df = pd.DataFrame(roundsList)
    try:
        #load them
        shows_df = pd.read_csv(os.path.join('data', 'shows.csv'))
        rounds_df = pd.read_csv(os.path.join('data', 'rounds.csv'))
        
        if str(show['Start Time'])[:21] in [tm[:21] for tm in shows_df['Start Time'].tolist()]:
            return False
        
        # append
        shows_df = shows_df.append(new_shows_df, ignore_index=True)
        rounds_df = rounds_df.append(new_rounds_df, ignore_index=True)
    except: # first time
        shows_df = new_shows_df
        rounds_df = new_rounds_df
        print('first time...creating csvs')
    
    # write them to their respective files
    shows_df.to_csv(os.path.join('data', 'shows.csv'), index=False)
    rounds_df.to_csv(os.path.join('data', 'rounds.csv'), index=False)
    return True

In [4]:
with open(os.path.join('C:\\Users','Joseph','AppData','LocalLow','Mediatonic','FallGuys_client','Player.log')) as f:
    lines = f.read()
#with open(os.path.join('C:\\Users','Joseph','Downloads','Player-prev.log')) as f:
 #   lines = f.read()

lines = lines.split('\n')
lines = preprocessGrade2(lines)

19319
18594


In [5]:
# get first line of each new show and usernames used for show
prevUser = '!!!!!!!!!!!!!!!'
lookUser = True
inRound = False
inARound = False
gameMode = 'main_show'

episodeMarkers = []
usernames = []
partySizes = []

# times
reg = []
conne = []
startRoundLines = []
userEndRoundLines = []
actualEndRoundLines = []

possLines = []
gameModes = []
# to find the actual number of players that qualified (for racing rounds)
prevNumLine = ""
numLines = []

# **********************************************************
# go through lines, looking for certain things**************
# **********************************************************
for i, line in enumerate(lines):
    # for username
    if lookUser and 'Sending login request' in line:
        usernames.append(line.split(' player ')[-1].split(' networkID')[0].replace(',', ''))
        prevUser = usernames[-1]
        lookUser = False
    # for type of show (main or alternate) (also called playlist)
    elif 'Chosen Show:' in line: # appears before entering matchmaking solo/group
        gameMode = line.split(':')[-1]
    # signifies start of looking for new episode
    # get size of party
    elif 'Party Size' in line or 'Begin matchmaking solo' in line:
        gameModes.append(gameMode)
        if 'Begin matchmaking solo' in line:
            partySize = 1
        else:
            partySize = int(line.split(' ')[-1].strip())
    # for show start time
    elif '[QosManager] Registered' in line or 'QosManager: Registered' in line: # for registered time (date)
        reg.append(line)
    elif "[StateConnectToGame] We're connected to the server!" in line: # for connection time
        conne.append(line)
    # for server ID and map lines
    elif 'Received NetworkGameOptions from ' in line: 
        tmp = line.split('roundID=')[-1]
        serverID = line.split(' ')[4]
        if 'Default' not in tmp:
            possLines.append([serverID, tmp])
    # for start round times and players that qualified from previous round
    elif 'state from Countdown to Playing' in line:
        startRoundLines.append(line.split(': [')[0])
        inRound = True
        inARound = True
        # append last # players achieving obj when hit new round
        if prevNumLine != "":
            numLines.append(prevNumLine.split('=')[-1])
            prevNumLine = ""
    elif '[ClientGameSession] NumPlayersAchievingObjective=' in line: # for total number of players that quality
        prevNumLine = line
    # for end round / player active in round times
    elif '[ClientGameManager] Handling unspawn for player FallGuy' in line and prevUser in line:
        if inRound:
            userEndRoundLines.append(line.split(': [')[0])
            inRound = False
    elif 'Changing local player state to: SpectatingEliminated' in line:
        if inRound:
            userEndRoundLines.append(line.split(': C')[0])
            inRound = False
        if inARound:
            actualEndRoundLines.append(line.split(': C')[0])
            inARound = False
    elif 'Changing state from Playing to GameOver' in line: # 'Changing state from GameOver to Results'
        if inARound:
            inARound = False
            actualEndRoundLines.append(line.split(': [')[0])
    # overall show data
    elif '[CompletedEpisodeDto]' in line: # marker for a good show
        partySizes.append(partySize)
        episodeMarkers.append(i)
        lookUser = True

# append last # achieving obj
numLines.append(prevNumLine.split('=')[-1])

# if no episodes found, end
if len(episodeMarkers) == 0:
    print('no episodes found') # change
        
# **********************************************************        
# get time user spent in each round ************************        
# (time round starts until they either finish or are eliminated) (just qualify I think)
# **********************************************************
roundTimes = []
for a, b in zip(startRoundLines, userEndRoundLines):
    roundTimes.append(subtractHours(a, b))

# get total round time
actualRoundTimes = []
for a, b in zip(startRoundLines, actualEndRoundLines):
    try:
        actualRoundTimes.append(subtractHours(a, b))
    except ValueError:
        actualRoundTimes.append('uncertain')

# gets start times for each show
startTimes = getStartTimes(reg, conne)


# **********************************************************
# for each show/episode ************************************
# **********************************************************
roundIdx = 0
for showIdx, (j, user) in enumerate(zip(episodeMarkers, usernames)):
    this_show = total_shows
    total_shows += 1
    
    # get lines for this show
    final_lines = getShowLines(lines, j)
    
    # split data
    showData, rounds = roundSplit(final_lines)
    
    # set show data
    show_dict = {}
    show_dict['Show ID'] = this_show # id
    show_dict['Start Time'] = startTimes[showIdx]
    show_dict['Season'] = getSeason() 
    show_dict['Time Taken'] = getTimeTaken(startTimes[showIdx], final_lines[0].split(': ==')[0]) # approximate time taken
    show_dict['Game Mode'] = gameModes[showIdx]
    show_dict['Final'] = rounds[-1][0].split('|')[-1].strip() in list_of_finals # final
    show_dict['Rounds'] = len(rounds) # num rounds
    show_dict['Username'] = user
    show_dict['Party Size'] = partySizes[showIdx]
    show_dict['addID'] = final_lines[0].split(': ==')[0] # end time
    
    # add other show data
    for line in showData:
        show_dict[line.split(':')[0]] = line.split(':')[1].strip()
    
    # ********************************************
    # get data for each round in show ************
    # ********************************************
    rounds_list = []
    rnds = getExtraRoundInfoLines(possLines)
    
    # for each round in the show/episode
    for round_ in rounds: # for list in 2D list
        round_dict = {'Show ID': this_show, 
                      'Round Num': round_[0].split(' ')[1].strip(), 
                      'Map': round_[0].split('|')[1].strip()}
        round_dict['Time Spent'] = roundTimes[roundIdx]
        round_dict['Round Length'] = actualRoundTimes[roundIdx]
        
        # add rest of data from list
        for line in round_[1:]:
            round_dict[line.split(':')[0]] = line.split(':')[1].strip()
        
        # add extra information
        rnd = rnds[roundIdx]
        splts = rnd.split()
        round_dict['Participants'] = splts[4].split('=')[-1] # num people
        round_dict['Qualification Percent'] = splts[7].split('=')[-1].replace(',', '') # qual %
        round_dict['Actual Num Qual'] = numLines[roundIdx]
        
        roundIdx += 1
        rounds_list.append(round_dict)
    
    # ********************************************
    # save ***************************************
    # ******************************************** 
    # save show_dict to one table
    # save each dict in rounds_list to another table
    if not saveData(show_dict, rounds_list):
        # print('already in csvsv')
        total_shows -= 1
    else:
        pass # print('saved')

print('csvs saved successfully')

with open('totalshows.txt', 'w') as f:
    f.write(str(total_shows))

csvs saved successfully


In [6]:
# see how a win looks

# get script to run when Player.log is updated

In [7]:
for line in lines:
    if 'Party Size' in line or 'Begin matchmaking solo' in line:
        print(line)
    if '[Round' in line:
        print(line)
    if '[CompletedEpisodeDto]' in line:
        print(line)

22:25:07.538: Party Size is 2
22:28:38.985: == [CompletedEpisodeDto] ==
[Round 0 | round_gauntlet_04]
[Round 1 | round_tunnel]
22:29:07.289: Party Size is 2
22:32:28.626: Party Size is 2
22:43:41.382: == [CompletedEpisodeDto] ==
[Round 0 | round_gauntlet_03]
[Round 1 | round_gauntlet_04]
[Round 2 | round_hoops_blockade_solo]
[Round 3 | round_block_party]
22:44:19.906: Party Size is 2
22:55:32.293: == [CompletedEpisodeDto] ==
[Round 0 | round_biggestfan]
[Round 1 | round_egg_grab_02]
[Round 2 | round_tip_toe]
[Round 3 | round_hoops_blockade_solo]


In [8]:
shows_df = pd.read_csv(os.path.join('data', 'shows.csv'))
shows_df

,Show ID,Start Time,Season,Time Taken,Game Mode,Final,Rounds,Username,Party Size,addID,Kudos,Fame,Crowns
0,367,2020-11-15 16:25:24.696,2,03:14.289000,main_show,False,2,Fast Swimming Panther,2,22:28:38.985,85,43,0
1,368,2020-11-15 16:29:38.367000,2,14:03.015000,main_show,False,4,Fast Swimming Panther,2,22:43:41.382,300,136,0
2,369,2020-11-15 16:32:47.769000,2,22:44.524000,main_show,False,4,Fast Swimming Panther,2,22:55:32.293,265,118,0


In [9]:
rounds_df = pd.read_csv(os.path.join('data', 'rounds.csv'))
rounds_df

,Show ID,Round Num,Map,Time Spent,Round Length,Qualified,Position,Kudos,Fame,Bonus Tier,Bonus Kudos,Bonus Fame,BadgeId,Participants,Qualification Percent,Actual Num Qual,Team Score
0,367,0,round_gauntlet_04,0:01:02.120000,0:01:28.055000,True,16,30,15,2.0,35,18,bronze,60,73,43,NaN
1,367,1,round_tunnel,0:00:14.287000,0:00:14.289000,False,39,20,10,NaN,0,0,NaN,43,61,0,NaN
2,368,0,round_gauntlet_03,0:00:51.486000,0:00:51.570000,True,25,30,15,2.0,35,18,bronze,60,73,43,NaN
3,368,1,round_gauntlet_04,0:01:22.753000,0:01:37.069000,True,5,20,10,1.0,70,35,silver,60,73,43,NaN
4,368,2,round_hoops_blockade_solo,0:00:51.137000,0:01:30.104000,True,7,60,20,2.0,35,18,bronze,41,73,29,NaN
5,368,3,round_block_party,0:01:37.755000,0:02:52.324000,False,14,50,20,NaN,0,0,NaN,29,60,17,NaN
6,369,0,round_biggestfan,0:01:41.687000,0:01:41.690000,True,10,30,15,1.0,70,35,silver,17,0,0,NaN
7,369,1,round_egg_grab_02,0:01:48.955000,0:03:30.039000,True,5,20,10,1.0,35,18,silver,60,73,36,12.0
8,369,2,round_tip_toe,0:02:06.260000,0:01:59.992000,True,13,60,20,NaN,0,0,NaN,36,100,0,NaN
9,369,3,round_hoops_blockade_solo,0:00:47.119000,0:00:47.387000,False,9,50,20,NaN,0,0,NaN,24,60,14,NaN


In [10]:
roundCodeDict = {
    'round_tip_toe': 'Tip Toe',
    'round_block_party': 'Block Party',
    'round_tunnel': 'Roll Out', # ???????????
    'round_egg_grab_02': 'Big Egg', #??
    'round_dodge_fall': 'Fruit Chute',
    'round_jump_showdown': 'Jump Showdown',
    
}

showTypeDict = {
    'main_show': 'Main Show',
    'event_only_survival_3010_to_0511': 'Slime Survivors',
    'event_only_season_2_variation_1011_to_1611': 'Medieval Mix Up',
}